#### Import libraries

In [146]:
import json
import pandas as pd
from pandas import json_normalize
import os
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

In [147]:
! pwd

/home/jovyan/work


#### Q1. Visualize the complete historical table view of each tables in tabular format in stdout (hint: print your table)

In [148]:
input_data = "/home/jovyan/work/data"
folder_list = os.listdir(input_data)
for folder_name in folder_list:
    temp_df = pd.DataFrame()

    sorted_child_filename_list = sorted(os.listdir(input_data+"/"+folder_name))
    for filename in sorted_child_filename_list:
        with open(input_data+ "/"+folder_name+ "/"+ filename) as json_data:
            obj = json.load(json_data)
            if obj['op'] == 'c':
                df = json_normalize(obj['data'])
                df["global_"+folder_name+"_id"] = obj["id"]
                df["ts"] = obj["ts"]
                temp_df = pd.concat([temp_df, df])
            if obj['op'] == 'u':
                last_filtered_account = temp_df[temp_df["global_"+folder_name+'_id'] == obj['id']].tail(1)
                for key, value in obj["set"].items():
                    #if value == "":
                    #    last_filtered_account[key] = np.nan
                    #else:
                    #   last_filtered_account[key] = value
                    last_filtered_account[key] = value

                last_filtered_account["ts"] = obj["ts"]
                temp_df = pd.concat([temp_df, last_filtered_account])
    
            
    # Translate back to df name
    if folder_name == "accounts":
        accounts_df = temp_df 
    elif folder_name == "cards":
        cards_df = temp_df 
    elif folder_name == "savings_accounts":
        savings_accounts_df = temp_df 

In [149]:
accounts_df

,account_id,name,address,phone_number,email,global_accounts_id,ts,savings_account_id,card_id
0,a1,Anthony,New York,12345678,anthony@somebank.com,a1globalid,1577863800000,NaN,NaN
0,a1,Anthony,New York,87654321,anthony@somebank.com,a1globalid,1577865600000,NaN,NaN
0,a1,Anthony,New York,87654321,anthony@somebank.com,a1globalid,1577890800000,sa1,NaN
0,a1,Anthony,Jakarta,87654321,anthony@anotherbank.com,a1globalid,1577894400000,sa1,NaN
0,a1,Anthony,Jakarta,87654321,anthony@anotherbank.com,a1globalid,1577926800000,sa1,c1
0,a1,Anthony,Jakarta,87654321,anthony@anotherbank.com,a1globalid,1579078860000,sa1,
0,a1,Anthony,Jakarta,87654321,anthony@anotherbank.com,a1globalid,1579163400000,sa1,c2


In [150]:
cards_df

,card_id,card_number,credit_used,monthly_limit,status,global_cards_id,ts
0,c1,11112222,0,30000,PENDING,c1globalid,1577926800000
0,c1,11112222,0,30000,ACTIVE,c1globalid,1578159000000
0,c1,11112222,12000,30000,ACTIVE,c1globalid,1578313800000
0,c1,11112222,19000,30000,ACTIVE,c1globalid,1578420000000
0,c1,11112222,0,30000,ACTIVE,c1globalid,1578654000000
0,c1,11112222,0,30000,CLOSED,c1globalid,1579078800000
0,c2,12123434,0,70000,PENDING,c2globalid,1579163400000
0,c2,12123434,0,70000,ACTIVE,c2globalid,1579298400000
0,c2,12123434,37000,70000,ACTIVE,c2globalid,1579361400000


In [151]:
savings_accounts_df

,savings_account_id,balance,interest_rate_percent,status,global_savings_accounts_id,ts
0,sa1,0,1.5,ACTIVE,sa1globalid,1577890800000
0,sa1,15000,1.5,ACTIVE,sa1globalid,1577955600000
0,sa1,15000,3.0,ACTIVE,sa1globalid,1578159060000
0,sa1,40000,3.0,ACTIVE,sa1globalid,1578648600000
0,sa1,21000,3.0,ACTIVE,sa1globalid,1578654000000
0,sa1,21000,1.5,ACTIVE,sa1globalid,1579078860000
0,sa1,21000,4.0,ACTIVE,sa1globalid,1579298460000
0,sa1,33000,4.0,ACTIVE,sa1globalid,1579505400000


#### Q2. Visualize the complete historical table view of the denormalized joined table in stdout by joining these three tables (hint: the join key lies in the resources section, please read carefully)

In [152]:
# add change account trigger
accounts_df['next_name']=accounts_df['name'].shift(1)
accounts_df['next_address']=accounts_df['address'].shift(1)
accounts_df['next_phone_number']=accounts_df['phone_number'].shift(1)
accounts_df['next_email']=accounts_df['email'].shift(1)
accounts_df['change_account'] = np.where((accounts_df['next_name']!=accounts_df['name']) | (accounts_df['next_address']!=accounts_df['address']) | (accounts_df['next_phone_number']!=accounts_df['phone_number']) | (accounts_df['next_email']!=accounts_df['email']),1,0)

In [153]:
# get next timestamp
accounts_df['next_ts']=accounts_df['ts'].shift(-1)
accounts_df['next_ts']=accounts_df['next_ts'].fillna(99999999999999999999)

cards_df['next_ts_card']=cards_df['ts'].shift(-1)
cards_df['next_ts_card']=cards_df['next_ts_card'].fillna(99999999999999999999)

savings_accounts_df['next_ts_saving']=savings_accounts_df['ts'].shift(-1)
savings_accounts_df['next_ts_saving']=savings_accounts_df['next_ts_saving'].fillna(99999999999999999999)

In [154]:
# account without card and saving
accounts_without_saving_card= accounts_df[accounts_df["card_id"].isnull()&accounts_df["savings_account_id"].isnull()]
accounts_without_saving_card['card_number']=np.nan
accounts_without_saving_card['credit_used']=np.nan
accounts_without_saving_card['monthly_limit']=np.nan
accounts_without_saving_card['card_status']=np.nan
accounts_without_saving_card['balance']=np.nan
accounts_without_saving_card['interest_rate_percent']=np.nan
accounts_without_saving_card['saving_status']=np.nan
accounts_without_saving_card_new = accounts_without_saving_card[["account_id", "name","address","phone_number","email","savings_account_id","card_id","card_number","credit_used","monthly_limit","card_status","balance","interest_rate_percent","saving_status","ts"]]

In [155]:
# join account with card
accounts_card_df = pd.merge(accounts_df, cards_df,  how='inner', on=['card_id'])
accounts_card_df.rename(columns = {'status':'card_status'}, inplace = True)

# filter by card
accounts_card_df_1=accounts_card_df.query('ts_y >= ts_x & ts_y < next_ts')
accounts_card_df_1['balance']=np.nan
accounts_card_df_1['interest_rate_percent']=np.nan
accounts_card_df_1['saving_status']=np.nan
accounts_card_df_new=accounts_card_df_1[["account_id", "name","address","phone_number","email","savings_account_id","card_id","card_number","credit_used","monthly_limit","card_status","balance","interest_rate_percent","saving_status","ts_y"]]
accounts_card_df_new.rename(columns = {'ts_y':'ts'}, inplace = True)

In [156]:
# join account with saving
accounts_saving_df = pd.merge(accounts_df, savings_accounts_df,  how='inner', on=['savings_account_id'])
accounts_saving_df.rename(columns = {'status':'saving_status'}, inplace = True)

# filter by saving
accounts_saving_df_1= accounts_saving_df.query('ts_y >= ts_x & ts_y < next_ts')
accounts_saving_df_1['card_number']=np.nan
accounts_saving_df_1['credit_used']=np.nan
accounts_saving_df_1['monthly_limit']=np.nan
accounts_saving_df_1['card_status']=np.nan
accounts_saving_df_new= accounts_saving_df_1[["account_id", "name","address","phone_number","email","savings_account_id","card_id","card_number","credit_used","monthly_limit","card_status","balance","interest_rate_percent","saving_status","ts_y"]]
accounts_saving_df_new.rename(columns = {'ts_y':'ts'}, inplace = True)

In [157]:
# filter by account change
accounts_df_cxg = accounts_df.query("change_account==1")
accounts_df_cxg = accounts_df_cxg[accounts_df_cxg["card_id"].notnull()|accounts_df_cxg["savings_account_id"].notnull()]
accounts_df_cxg = accounts_df_cxg[["account_id", "name","address","phone_number","email","savings_account_id","card_id","ts"]]

# find card infor change
accounts_df_cxg_card = pd.merge(accounts_df_cxg, cards_df,  how='left', on=['card_id'])
accounts_df_cxg_card['balance']=np.nan
accounts_df_cxg_card['interest_rate_percent']=np.nan
accounts_df_cxg_card['saving_status']=np.nan
accounts_df_cxg_card= accounts_df_cxg_card.query('ts_y < ts_x & ts_x < next_ts_card')
accounts_df_cxg_card.rename(columns = {'status':'card_status'}, inplace = True)
accounts_df_cxg_card = accounts_df_cxg_card[["account_id", "name","address","phone_number","email","savings_account_id","card_id","card_number","credit_used","monthly_limit","card_status","balance","interest_rate_percent","saving_status","ts_x"]]
accounts_df_cxg_card.rename(columns = {'ts_x':'ts'}, inplace = True)

# find saving infor change
accounts_df_cxg_saving = pd.merge(accounts_df_cxg, savings_accounts_df,  how='left', on=['savings_account_id'])
accounts_df_cxg_saving= accounts_df_cxg_saving.query('ts_y < ts_x & ts_x < next_ts_saving')
accounts_df_cxg_saving['card_number']=np.nan
accounts_df_cxg_saving['credit_used']=np.nan
accounts_df_cxg_saving['monthly_limit']=np.nan
accounts_df_cxg_saving['card_status']=np.nan
accounts_df_cxg_saving.rename(columns = {'status':'saving_status'}, inplace = True)
accounts_df_cxg_saving= accounts_df_cxg_saving[["account_id", "name","address","phone_number","email","savings_account_id","card_id","card_number","credit_used","monthly_limit","card_status","balance","interest_rate_percent","saving_status","ts_x"]]
accounts_df_cxg_saving.rename(columns = {'ts_x':'ts'}, inplace = True)


In [158]:
# merge
frames = [accounts_card_df_new, accounts_saving_df_new,accounts_without_saving_card_new,accounts_df_cxg_card,accounts_df_cxg_saving ]
df_result = pd.concat(frames)
df_result = df_result.sort_values(by=['ts'])

In [159]:
#Forward fill data but if card_id or savings_account_id is set to "", we dont need to forward fill the data, just let other appropriate field as ""
df_result = df_result.fillna(method='ffill')
df_result.loc[df_result['card_id'] == "", ['card_number', 'credit_used', 'monthly_limit', 'card_status']] = ""
df_result.loc[df_result['savings_account_id'] == "", ['balance', 'interest_rate_percent', 'saving_status']] = ""

In [160]:
# If they have same timestamp, take the latest one in the group, we have forward fill based on ts so it should be correct
df_result = df_result.groupby('ts').tail(1)
df_result


,account_id,name,address,phone_number,email,savings_account_id,card_id,card_number,credit_used,monthly_limit,card_status,balance,interest_rate_percent,saving_status,ts
0,a1,Anthony,New York,12345678,anthony@somebank.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1577863800000
0,a1,Anthony,New York,87654321,anthony@somebank.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1577865600000
0,a1,Anthony,New York,87654321,anthony@somebank.com,sa1,NaN,NaN,NaN,NaN,NaN,0.0,1.5,ACTIVE,1577890800000
0,a1,Anthony,Jakarta,87654321,anthony@anotherbank.com,sa1,NaN,NaN,NaN,NaN,NaN,0.0,1.5,ACTIVE,1577894400000
0,a1,Anthony,Jakarta,87654321,anthony@anotherbank.com,sa1,c1,11112222,0.0,30000.0,PENDING,0.0,1.5,ACTIVE,1577926800000
17,a1,Anthony,Jakarta,87654321,anthony@anotherbank.com,sa1,c1,11112222,0.0,30000.0,PENDING,15000.0,1.5,ACTIVE,1577955600000
1,a1,Anthony,Jakarta,87654321,anthony@anotherbank.com,sa1,c1,11112222,0.0,30000.0,ACTIVE,15000.0,1.5,ACTIVE,1578159000000
18,a1,Anthony,Jakarta,87654321,anthony@anotherbank.com,sa1,c1,11112222,0.0,30000.0,ACTIVE,15000.0,3.0,ACTIVE,1578159060000
2,a1,Anthony,Jakarta,87654321,anthony@anotherbank.com,sa1,c1,11112222,12000.0,30000.0,ACTIVE,15000.0,3.0,ACTIVE,1578313800000
3,a1,Anthony,Jakarta,87654321,anthony@anotherbank.com,sa1,c1,11112222,19000.0,30000.0,ACTIVE,15000.0,3.0,ACTIVE,1578420000000


#### Q3. From result from point no 2, discuss how many transactions has been made, when did each of them occur, and how much the value of each transaction?
#### Transaction is defined as activity which change the balance of the savings account or credit used of the card

In [161]:
transactions_df = df_result[["ts","credit_used","balance"]]
transactions_df['diff_credit_used'] = transactions_df['credit_used'].replace("", 0).fillna(0).astype("Int32").diff()
transactions_df['diff_balance'] = transactions_df['balance'].replace("", 0).fillna(0).astype("Int32").diff()
transactions_df

,ts,credit_used,balance,diff_credit_used,diff_balance
0,1577863800000,NaN,NaN,<NA>,<NA>
0,1577865600000,NaN,NaN,0,0
0,1577890800000,NaN,0.0,0,0
0,1577894400000,NaN,0.0,0,0
0,1577926800000,0.0,0.0,0,0
17,1577955600000,0.0,15000.0,0,15000
1,1578159000000,0.0,15000.0,0,0
18,1578159060000,0.0,15000.0,0,0
2,1578313800000,12000.0,15000.0,12000,0
3,1578420000000,19000.0,15000.0,7000,0


In [162]:
filtered_transactions_df = transactions_df[(transactions_df['diff_credit_used'] != 0.0) | (transactions_df['diff_balance'] != 0.0) ]
filtered_transactions_df = filtered_transactions_df[filtered_transactions_df[['diff_credit_used', 'diff_balance']].notnull().all(1)]
filtered_transactions_df
# 8 transactions happening, the "diff_credit_used" and "diff_balance" show the value of the transactions. 
# "credit_used" and "balance" is what the user currently have on that timestamp
# at TS: 1578654000000, there are two transaction going on

,ts,credit_used,balance,diff_credit_used,diff_balance
17,1577955600000,0.0,15000.0,0,15000
2,1578313800000,12000.0,15000.0,12000,0
3,1578420000000,19000.0,15000.0,7000,0
19,1578648600000,19000.0,40000.0,0,25000
20,1578654000000,0.0,21000.0,-19000,-19000
8,1579361400000,37000.0,21000.0,37000,0
39,1579505400000,37000.0,33000.0,0,12000
